In [1]:
%pwd

'/home/aistudio'

In [2]:
# 测试paddle conv2d 参数;  
# 完成 整个 conv+pool block;  finish 测试
# 本周开始搭建视觉模型；paper list;
# 整体训练框架;
# alex next 
# vgg 
# googleNet 
# resnet 
# inception 
# dpn 
# SE-ResNeXt 
# https://www.cnblogs.com/vincent1997/p/10901875.html
# https://cloud.tencent.com/developer/article/1382720
# https://blog.csdn.net/hua111hua/article/details/89422661#PaddlePaddleVisualDL_132 可视化
# https://aistudio.baidu.com/bdvgpu/user/127073/136105/visualdl/static/index.html 最后几项改成这个

In [3]:
import sys
sys.path.append('/home/aistudio/work')
from models.model import conv_pool_layer, alex_net
from models.utils import load_image, to_rgb_img
from models.data import image_reader_creator, train_valid_split

from visualdl import LogWriter
import paddle.fluid as fluid
import paddle
from paddle.fluid.initializer import TruncatedNormalInitializer
import numpy as np
import sys
import os
from glob import glob
from multiprocessing import cpu_count
import matplotlib.pyplot as pyplot
import subprocess

In [4]:
gt_dir = '/home/aistudio/work/task3/train/truth_pic/'
fk_dir = '/home/aistudio/work/task3/train/rumor_pic/'
revert_gt_dir = '/home/aistudio/work/task3/train/truth_revert/'
revert_fk_dir = '/home/aistudio/work/task3/train/fake_revert/'
truth_samples = [(i, 1) for i in glob(gt_dir+'*')[10:20]]
fake_samples = [(i, 0) for i in glob(fk_dir+'*')]
truth_revert_samples = [(i, 1) for i in glob(revert_gt_dir+'*')]
fake_revert_samples = [(i, 0) for i in glob(revert_fk_dir+'*')]
all_samples = [sample for sub_lis in [truth_samples, fake_samples, truth_revert_samples, fake_revert_samples] for sample in sub_lis]
num_cls = 2
EPOCH = 100
bs = 400
buf_size = 4000

In [6]:
main_program = fluid.Program()
startup_program = fluid.Program()
with fluid.program_guard(main_program=main_program, startup_program=startup_program):
    data = fluid.layers.data(name='data', shape=[3,100,100], dtype='float32')
    label = fluid.layers.data(name="label", shape=[1], dtype='int64')
    net_out = alex_net(data)
    pred = fluid.layers.fc(input=net_out, size=num_cls, act='softmax',
                            param_attr=paddle.fluid.ParamAttr(name="param1", initializer=TruncatedNormalInitializer()))
    
    loss = fluid.layers.cross_entropy(input=pred, label=label)
    ave_loss = fluid.layers.mean(loss)

    # batch_size = fluid.layers.create_tensor(dtype='int64')
    # print('bs:{}'.format(batch_size))
    
    acc = fluid.layers.accuracy(input=pred, label=label)
    auc,_,_ = fluid.layers.auc(input=pred, label=label)
    test_program = main_program.clone(for_test=True)
    sgd = fluid.optimizer.AdamOptimizer(learning_rate=0.001)
    sgd.minimize(ave_loss)
        

chnnels:3
chnnels:64
chnnels:192
chnnels:384
conv4 shape:(-1, 256, 3, 3)


In [7]:
logwriter = LogWriter("./log", sync_cycle=10)

with logwriter.mode("train") as writer:
    train_loss_scalar = writer.scalar("loss")

with logwriter.mode("train") as writer:
    train_acc_scalar = writer.scalar("acc")

with logwriter.mode("train") as writer:
    train_acc2_scalar = writer.scalar("acc2")
    
with logwriter.mode("train") as writer:
    train_auc_scalar = writer.scalar("auc")

with logwriter.mode("train") as writer:
    train_auc2_scalar = writer.scalar("auc2")

with logwriter.mode("test") as writer:
    test_loss_scalar = writer.scalar("loss")

with logwriter.mode("test") as writer:
    test_acc_scalar = writer.scalar("acc")

with logwriter.mode("test") as writer:
    test_acc2_scalar = writer.scalar("acc2")
    
with logwriter.mode("test") as writer:
    test_auc_scalar = writer.scalar("auc")

with logwriter.mode("test") as writer:
    test_auc2_scalar = writer.scalar("auc2")
    


In [8]:
train, test = train_valid_split(all_samples, 0.8)
len([i for i in test if i[1]==1])

8

In [9]:
use_cuda = True
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()
exe = fluid.Executor(place)

train_reader = image_reader_creator(train, len(train))
test_reader = image_reader_creator(test, len(test))
batch_train_reader = paddle.batch(train_reader, bs)
batch_test_reader = paddle.batch(test_reader, bs)

feeder = fluid.DataFeeder(place=place, feed_list=[data, label])

train_step = 0
test_step = 0

exe.run(startup_program)

train_acc_metric = fluid.metrics.Accuracy()
train_auc_metric = fluid.metrics.Auc("ROC")
test_acc_metric = fluid.metrics.Accuracy()
test_auc_metric = fluid.metrics.Auc("ROC")


for epoch in range(EPOCH):
    for bs_data in batch_train_reader():
        tra_pred, tra_label, tra_loss, tra_acc, tra_auc = exe.run(main_program, feed=feeder.feed(bs_data),
                                                fetch_list=[pred, label, ave_loss, acc, auc])
           
        train_acc_metric.update(value=tra_acc, weight=bs)  
        train_auc_metric.update(tra_pred, tra_label)
        
        train_loss_scalar.add_record(train_step, tra_loss[0])
        train_acc_scalar.add_record(train_step, tra_acc[0])
        train_acc2_scalar.add_record(train_step, train_acc_metric.eval())
        train_auc_scalar.add_record(train_step, tra_auc[0])
        train_auc2_scalar.add_record(train_step, train_auc_metric.eval())
        train_step+=1


    for batch_id, data in enumerate(batch_test_reader()):
        test_pred, test_label, test_loss, test_acc, test_auc = exe.run(program=test_program, feed=feeder.feed(data),            
                                      fetch_list=[pred, label, ave_loss, acc, auc])     
        test_acc_metric.update(value=test_acc, weight=bs)  
        test_auc_metric.update(test_pred, test_label)
        
        test_loss_scalar.add_record(test_step, test_loss[0])
        test_acc_scalar.add_record(test_step, test_acc[0])
        test_acc2_scalar.add_record(test_step, test_acc_metric.eval())
        test_auc_scalar.add_record(test_step, test_auc[0])
        test_auc2_scalar.add_record(test_step, test_auc_metric.eval())
        test_step+=1


In [10]:
https://aistudio.baidu.com/bdvgpu/user/127073/136105/visualdl/static/index.html

In [11]:
https://aistudio.baidu.com/bdvgpu/user/127073/136105/visualdl/static/index.html

In [ ]:
https://aistudio.baidu.com/bdvgpu/user/127073/136105/notebooks/136105.ipynb?redirects=1